# setup

In [1]:
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse 

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130911 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# library

In [3]:
import os
os.chdir("./drive/My Drive")
os.chdir("./2019-International-Data-Science-Competition")
print(os.getcwd())

if not os.path.exists("./save_model"): os.mkdir("./save_model")
if not os.path.exists("./result"): os.mkdir("./result")

/content/drive/My Drive/2019-International-Data-Science-Competition


In [4]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from scipy.signal import medfilt

import time
import datetime
import os
import sys
from math import *

from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler
from keras import models
from keras import backend as K
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers import Input
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras import layers
from keras.models import model_from_json
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Input, Activation, Concatenate
from keras import metrics

scaler = StandardScaler()



Using TensorFlow backend.


# read file

In [5]:
traindf = pd.read_csv("data/Train_full.csv")
col = traindf.columns
print(len(traindf))

709314


# preprocessing

In [0]:
col_feature = ['body', 'Open', 'Close', 'High', 'Low',
               'SMA_50', 'SMA_20', 'ATR', 'CCI', 'SAR', 'hour', 'min', 'dayofweek', 
               'lag_return_1', 'lag_return_2', 'lag_return_3', 'lag_return_4',
               'lag_return_5', 'lag_return_6', 'lag_return_7', 'lag_return_8',
               'up_down']

col_sel = col_feature

lag = 12
BATCH_SIZE = 64
num_train = BATCH_SIZE * 5000 + lag
num_val = BATCH_SIZE * 50 + lag

In [0]:
train_data = traindf[col_sel].iloc[-num_train:, :].values.copy()
val_data = traindf[col_sel].iloc[:num_val, :].values.copy()

In [0]:
def create_data(data, lag=4, train_phase=True):
    if train_phase:
        X = np.zeros((data.shape[0]-lag, lag+1, data.shape[1]-1))
        begin = lag
        for i in range(begin, data.shape[0]):
            X[i-lag, :, :] = data[i-lag:i+1, :-1]
        Y = np.expand_dims(data[lag:, -1], axis=1)
        return X, Y

In [0]:
Xtrain, Ytrain = create_data(train_data, lag=lag, train_phase=True)
Xval, Yval = create_data(val_data, lag=lag, train_phase=True)

# model

In [0]:
def define_model(batch_size, time_step, n_features):
    model = models.Sequential()

    model.add(layers.LSTM(300, activation='tanh', batch_input_shape=(batch_size, time_step, n_features), return_sequences=True))
    model.add(Dropout(0.2))
    
    model.add(layers.LSTM(250, activation='tanh',  return_sequences=True))
    model.add(Dropout(0.2))
    
    model.add(layers.LSTM(200, activation='relu',  return_sequences=True))
    model.add(Dropout(0.2))
    
    model.add(layers.LSTM(150, activation='tanh',  return_sequences=True))
    model.add(Dropout(0.2))

    model.add(layers.LSTM(100, activation='relu',  return_sequences=True))
    model.add(Dropout(0.2))

    model.add(layers.Flatten())
    
    model.add(layers.Dense(1, activation = "sigmoid"))
    
    return model

def fit_lstm(Xtrain, Ytrain, Xval, Yval):
    tic = time.time()
    batch_size = BATCH_SIZE
    nb_epoch = 20
    
    model = define_model(batch_size=batch_size, time_step=Xtrain.shape[1], n_features=Xtrain.shape[2])
    """
    if os.path.exists("./save_model/model.json"):
        print("Loaded model from disk")
        json_file = open('./save_model/model.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        # load weights into new model
        model.load_weights("./save_model/model.h5")
    """
    #model.summary()
    
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=5e-4), metrics = [metrics.binary_accuracy])
    
    WEIGHTS_BEST = "./save_model/best_model.hdf5"
    early_stoping = EarlyStopping(monitor='val_binary_accuracy', patience=2, verbose=1)
    save_checkpoint = ModelCheckpoint(WEIGHTS_BEST, monitor = 'val_binary_accuracy', verbose = 1, save_best_only = True, save_weights_only = True, mode='max')
    reduce_lr = ReduceLROnPlateau(monitor = 'val_binary_accuracy', factor = 0.1, patience = 2, min_lr = 1e-8, verbose=1)
    callbacks = [early_stoping, reduce_lr, save_checkpoint]
    
    model.fit(Xtrain, Ytrain, 
              epochs=nb_epoch, 
              batch_size=batch_size,
              validation_data = (Xval, Yval),
              verbose=1,
              callbacks=callbacks)
    
    # serialize model to JSON
    model_json = model.to_json()
    with open("./save_model/model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("./save_model/model.h5")
    print("Saved model to disk")
    
    toc = time.time()
    print("TRAINING TIME: ", toc-tic)
#     return model

# train

In [54]:
fit_lstm(Xtrain, Ytrain, Xval, Yval)

Train on 320000 samples, validate on 3200 samples
Epoch 1/20
320000/320000 [==============================] - 434s 1ms/step - loss: 0.5075 - binary_accuracy: 0.6574 - val_loss: 0.5373 - val_binary_accuracy: 0.6356

Epoch 00001: val_binary_accuracy improved from -inf to 0.63562, saving model to ./save_model/best_model.hdf5
Epoch 2/20
320000/320000 [==============================] - 422s 1ms/step - loss: 0.4997 - binary_accuracy: 0.6659 - val_loss: 0.5357 - val_binary_accuracy: 0.6397

Epoch 00002: val_binary_accuracy improved from 0.63562 to 0.63969, saving model to ./save_model/best_model.hdf5
Epoch 3/20
320000/320000 [==============================] - 422s 1ms/step - loss: 0.4974 - binary_accuracy: 0.6681 - val_loss: 0.5360 - val_binary_accuracy: 0.6472

Epoch 00003: val_binary_accuracy improved from 0.63969 to 0.64719, saving model to ./save_model/best_model.hdf5
Epoch 4/20
320000/320000 [==============================] - 422s 1ms/step - loss: 0.4963 - binary_accuracy: 0.6686 - val_l

In [0]:
model = define_model(batch_size=BATCH_SIZE, time_step=Xtrain.shape[1], n_features=Xtrain.shape[2])
model.load_weights("save_model/best_model.hdf5")

# validate & test

In [0]:
testdf = pd.read_csv("data/Test_small_features.csv")

In [0]:
test_data = testdf[col_sel[:-1]].iloc[:, :].values.copy()
test_data = np.r_[train_data[-lag:, :-1], test_data]
test_data = np.c_[test_data, [0]*test_data.shape[0]]

Xtest, Ytest = create_data(test_data, lag=lag, train_phase=True)

In [0]:
end_b = Xtest.shape[0] - int(Xtest.shape[0]/BATCH_SIZE)*BATCH_SIZE
y_pd1 = model.predict(Xtest[:-end_b], batch_size=BATCH_SIZE)
y_pd2 = model.predict(Xtest[-BATCH_SIZE:], batch_size=BATCH_SIZE)

In [59]:
y_pd = np.r_[y_pd1, y_pd2[-end_b:]].flatten()

yhat = (y_pd>0.5).astype(int)
print(sum(yhat) / len(yhat))

0.18153621601897463


# save file

In [0]:
submit = pd.DataFrame({"index":[*range(len(yhat))], "label":yhat})
submit.to_csv("result/submit_final.csv", index=None)